In [1]:
import pyltp
from gensim.models import Word2Vec
from pyltp import Segmentor
import jieba
from gensim.models.word2vec import LineSentence
from pyltp import  SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
from gensim import models
import numpy as np
from gensim import corpora

In [23]:
import pandas as pd
import jieba
import re
import math
import numpy as np
import random
import gensim


In [4]:
data_source = '/home/kg/PycharmProjects/TextGrapher-master/ltp_data/sqlResult_1558435.csv'
data = pd.read_csv(data_source,encoding='UTF-8')

In [5]:
data = data.fillna('')

In [6]:
content = data['content'].tolist()

In [10]:
def token(string):
    return re.findall(r'[\d|\w]+',string)


In [8]:
def cut(string): return ' '.join(jieba.cut(string))

In [11]:
news_content = [token(n) for n in content]
news_content = [''.join(n) for n in news_content]

news_content = [cut(n) for n in news_content]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.920 seconds.
Prefix dict has been built succesfully.


In [12]:
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')

In [13]:
stop_words = []
with open('/home/kg/PycharmProjects/TextGrapher-master/ltp_data/百度停用词表.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [16]:
import jieba.posseg as jp,jieba

In [19]:
len(news_content)

89611

In [20]:
news_content=news_content[:200]

In [21]:
words_ls = []
for text in news_content:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

In [24]:
news_model = gensim.models.Word2Vec(words_ls, min_count=1)

In [39]:
def get_similar_words(ini_words, model=news_model, max_len=7):
    unseen = ini_words
    seen = defaultdict(int)
    
    while unseen and len(seen)<max_len:
        word = unseen.pop(0)
        
        similar_words = [w for w,_ in model.most_similar(word, topn=5)]
        
        unseen += similar_words
        
        seen[word] += 1
        
    return sorted(seen, key=lambda x:seen[x], reverse=True)[:7]

In [27]:
from collections import defaultdict

In [40]:
wl = ['说']
similar_say = [w for w in get_similar_words(wl) if w not in['了解','立场','估算','测算','统计','不禁','不已','一句','看来']]
print(similar_say)

['说', '还', '人', '会', '都', '记者', '中国']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [60]:
# get_similar_words 效果不太好

In [41]:
cws_model = "/home/kg/PycharmProjects/TextGrapher-master/ltp_data/ltp_data_v3.4.0/cws.model"
pos_model = "/home/kg/PycharmProjects/TextGrapher-master/ltp_data/ltp_data_v3.4.0/pos.model"
par_model = "/home/kg/PycharmProjects/TextGrapher-master/ltp_data/ltp_data_v3.4.0/parser.model"
ner_model = "/home/kg/PycharmProjects/TextGrapher-master/ltp_data/ltp_data_v3.4.0/ner.model"

In [42]:
def preprocess(text):
    text = text.replace(u'\r\n', u'')
    text = text.replace(u'\u3000', u'')
    text = text.replace(u'\n', u'')
    text = text.replace(u'\\r\\n', u'')
    text = text.replace(u'\\u3000', u'')
    text = text.replace(u'\\n', u'')
    return text

In [43]:
def split_sentences(text):
    #得到分句子
    st = ''
    sents = SentenceSplitter.split(text)
    sentences = [s for s in sents if len(s) != 0]
    sentences = [preprocess(s) for s in sentences]
    return sentences

def get_word_list(sentence,model=cws_model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model=pos_model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model=par_model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

def get_ner_list(word_list,postag_list,model=ner_model):
    #命名实体识别
    reg = NamedEntityRecognizer()                    #初始化命名实体实例
    reg.load(model)                                       #加载模型
    netags = reg.recognize(word_list, postag_list)         #对分词、词性标注得到的数据进行实体标识
    netags = list(netags)
    reg.release()
    return netags


In [57]:
def get_speech(text):
    ner_set = ['S-Ni', 'S-Ns', 'S-Nh', 'B-Ni', 'B-Ns', 'B-Nh', 'I-Ni', 'I-Ns', 'I-Nh', 'E-Ni', 'E-Ns', 'E-Nh']
    punc_set = ['，',',','：',';','；','。','!','！','？','?']

    sentences = split_sentences(text)
    words = [get_word_list(sent) for sent in sentences]
    postags = [get_postag_list(w) for w in words]
    parsers = [get_parser_list(w,p) for w, p in zip(words, postags)]
    ners = [get_ner_list(w,p) for w, p in zip(words, postags)]
    
    #取‘说’的近义词
    similar_say = ['表示', '说', '指出', '认为', '坦言', '告诉', '强调', '称', '直言', '普遍认为', '介绍', '透露', '重申', '呼吁', '说道', '感叹', '地说', '写道', '中称', '证实', '还称', '猜测', '暗示', '感慨', '热议', '敦促', '指责', '声称', '主张', '反对', '批评', '表态', '中说', '承认', '却说', '感触', '提到', '所说', '引述', '质疑', '抨击']
    similar_say += ['回应','分析']
    content = {}
    start = 0
    end = -1
    said = []
    for i in range(len(sentences)):
        if not sentences[i]:
            continue
        for j in range(len(words[i])):
            #主谓关系找主语和谓语为‘说’相近的动词的句子，这里只考虑主语为特定实体（人名、地名、机构名）或者一般性名词，
            if parsers[i][j][1]=='SBV' and (ners[i][j]!='O' or postags[i][j]=='n'):
                pos = parsers[i][j][0]-1

                #谓语动词为‘说’近义词或者谓语动词下一个词与谓语动词为并列关系且为‘说’的同义词
                if words[i][pos] in similar_say or '说' in words[i][pos] or (parsers[i][pos+1][1]=='COO' and parsers[i][pos+1][0]==(pos+1) and words[i][pos+1] in similar_say): 
                                    
                    #之前句子内的言论      
                    if ('”' in words[i-1]) and ('说' not in sentences[i-1]) and i>0:
                        m = words[i-1].index('”')
                        if ('“' in words[i-1]):
                            n = words[i-1].index('“')
                            if m-n >3:
                                said = said+words[i-1][n+1:m]
                        else:
                            for k in range(1,i-1):
                                if '“' in words[i-1-k]:
                                    n = words[i-1-k].index('“')
                                    said = said+words[i-1-k][n+1:]+words[i-1][:m]
                                    break
                    
                    #本句前面的言论
                    if ('”' in words[i][:pos]):
                        m = words[i].index('”')
                        if ('“' in words[i]):
                            n = words[i].index('“')
                            said = said+words[i][n+1:m]
                        else:
                            for k in range(1,i):
                                if '“' in words[i-k]:
                                    n = words[i-k].index('“')
                                    said = said+words[i-k][n+1:]+words[i][:m]
                                    break
                                    
                    #本句后面的言论
                    start = pos+1
                    if words[i][start] in punc_set:
                        start += 1
                    said = said+words[i][start:]
                      
                    #之后句子内的言论
                    if ('“' in words[i+1]) and ('说' not in sentences[i+1]) and i<len(sentences)-1:
                        m = words[i+1].index('“')
                        if ('”' in words[i+1]):
                            n = words[i+1].index('”')
                            if n-m >3: 
                                said = said+words[i+1][m+1:n]
                        else:
                            for k in range(i+2,len(sentences)):
                                if '”' in words[k]:
                                    n = words[k].index('”')
                                    said = said+words[i+1][m+1:] + words[k][:n]
                                    break

                    if ners[i][j]!='O' and postags[i][j]=='n':
                        name = words[i][j-1:j+1]
                    else:
                        name = words[i][j]
                    
                    if name in content.keys():# 同一人/机构/团体等的言论合并
                        content[name] += [''.join(said)]
                    else:
                        content[name] = [''.join(said)]
                        
                    said = []
                    
    return content

In [52]:
content[35]

'\u3000\u3000北京时间6月20日，据sportando报道，有CBA球队与上赛季效力于西班牙联赛的法国后卫埃德温-杰克逊联系到了一起，不过目前尚不清楚具体是哪家俱乐部。\r\n\u3000\u3000杰克逊的经纪人今天在推特上用中文写下了埃德温-杰克逊的名字，这被认为是在暗示该球员未来的去向。杰克逊也似乎在推特上向他的球迷告别，同时还表态称，虽然自己更愿意留在欧洲联赛或者去NBA发展，但他觉得这份合同（来CBA打球）仅仅是一份6个月的短合同，之后他还会有其他选择。\r\n\u3000\u3000埃德温-杰克逊生于1989年9月18日，身高190cm的他在场上司职得分后卫。上赛季埃德温-杰克逊在西甲联赛的大学生队效力期间，场均出场30.3分钟能够拿到21.4分，3.5个篮板，3.4次助攻以及1次抢断的数据。\r\n\u3000\u3000据欧洲篮球专家王健微博透露，这名法国后卫有超强的个人得分能力，堪称刷分机器。其个人能力与美籍后卫相比，也丝毫不逊色。\r\n\u3000\u3000（豪斯）\r\n'

In [59]:
 get_speech(content[35])

{'微博': ['这名法国后卫有超强的个人得分能力，堪称刷分机器。']}

In [54]:
test = '\\n\\n\\n新华社北京5月8日新媒体专电;英媒称，斯蒂芬·霍金教授警告说，人类如果想要生存下去，就必须在100年内逃离地球。\\n\\n\\n\\n据英国《每日邮报》网站5月3日报道，这位著名理论物理学家认为，地球上的生命被灾难毁灭的风险越来越大，包括小行星撞击、传染病、人口过多和气候变化等。\\n\\n\\n霍金教授在一部新纪录片中警告说，在今后一个世纪里，我们的世界会变得越来越不宜居，未来几代人必须在太空中寻找新的生存空间。\\n\\n\\n报道称，在新纪录片《远征新地球》中，霍金教授将周游世界，探讨人类如何在外太空生存。这部纪录片是英国广播公司（BBC）将于6月开播的《明日世界》节目的一部分。\\n\\n\\n在这部将由BBC二台播出的纪录片中，霍金声称人类能够在地球上生存的时间所剩无几，我们必须去其他地方探寻自己的未来。\\n\\n\\n报道称，去年11月，霍金对于自己的估计还比较保守。他当时曾警告说，人类无法在“脆弱”的地球上再生存1000年。\\n\\n\\n在那个谈话节目中，霍金用1小时的时间快速回顾了人类对宇宙起源的认识的发展历史，从最初的创世神话一直谈到了最先进的M理论预言。\\n\\n\\n他说：“也许有朝一日我们能够利用引力波来回看宇宙大爆炸的中心。”\\n\\n\\n“宇宙学的最新进展都是在太空中取得的，在那里可以不受影响地观测我们的宇宙，但我们继续探索太空也是为了人类的未来，”他说，“我认为，如果不逃离我们脆弱的星球，我们将无法再生存1000年。”\\n\\n\\n霍金接着说：“因此我希望激发公众对太空的兴趣，而我也一直在进行早期训练。”他曾说自己希望搭乘维珍集团老板理查德·布兰森的维珍银河飞船前往太空。\\n\\n\\n霍金还说：“能够活着并从事理论物理学研究，这真是一段美好的时光。在过去50年里，我们对宇宙的认识发生了巨大变化。如果我为此作出了一份小小贡献的话，那么我感到十分高兴。”\\n\\n\\n他说：“我们人类只是自然界基本粒子的组合，而我们能够如此接近于理解那些支配我们和宇宙的规律，这本身就是一项伟大的成就。”\\n\\n\\n报道称，霍金以前曾在《如何制造宇宙飞船》一书的编后语中谈过他对未来太空旅行的观点。\\n\\n'

In [58]:
 get_speech(test)

{'报道': ['在新纪录片《远征新地球》中，霍金教授将周游世界，探讨人类如何在外太空生存。',
  '去年11月，霍金对于自己的估计还比较保守。',
  '霍金以前曾在《如何制造宇宙飞船》一书的编后语中谈过他对未来太空旅行的观点。'],
 '教授': ['说，人类如果想要生存下去，就必须在100年内逃离地球。',
  '说，在今后一个世纪里，我们的世界会变得越来越不宜居，未来几代人必须在太空中寻找新的生存空间。'],
 '物理学家': ['地球上的生命被灾难毁灭的风险越来越大，包括小行星撞击、传染病、人口过多和气候变化等。'],
 '霍金': ['人类能够在地球上生存的时间所剩无几，我们必须去其他地方探寻自己的未来。',
  '“因此我希望激发公众对太空的兴趣，而我也一直在进行早期训练。”',
  '“能够活着并从事理论物理学研究，这真是一段美好的时光。']}